## 运行环境为python2

In [0]:
!pip list

Package                            Version              
---------------------------------- ---------------------
absl-py                            0.7.1                
aenum                              2.1.2                
alabaster                          0.7.12               
albumentations                     0.1.12               
altair                             3.1.0                
astor                              0.8.0                
astropy                            2.0.14               
atari-py                           0.1.7                
atomicwrites                       1.3.0                
attrs                              19.1.0               
audioread                          2.1.8                
autograd                           1.2                  
Babel                              2.7.0                
backports-abc                      0.5                  
backports.functools-lru-cache      1.5                  
backports.shutil-get-terminal-s

In [0]:
!pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp27-cp27mu-linux_x86_64.whl
!pip install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp27-cp27mu-linux_x86_64.whl

In [0]:
!git clone https://github.com/yinchangchang/ocr_densenet.git

In [0]:
!cd ./ocr_densenet;pip install -r requirement.txt

In [0]:
import pandas as pd
data = pd.read_csv('./ocr_densenet/files/train.csv')
data.head()

,name,content
0,GK5NXT2E.jpg,SJ88154371
1,PNFRISAL.jpg,XI90599371
2,2GVHF7RK.jpg,WG35669371
3,DGUVOETG.jpg,YH01075371
4,ARIQMNK3.jpg,SJ44155371


In [0]:
# !mkdir ./ocr_densenet/data
# !cp ./drive/my_gpu/tinymind/test_cptn_result.zip ./;unzip test_cptn_result.zip
!cp ./drive/my_gpu/tinymind/train_cptn_result.zip ./;unzip train_cptn_result.zip

###将模型copy到此目录下

In [0]:
!cd ocr_densenet/code/ocr;mkdir -p ../../data/models-small/densenet/eval-16-1/

In [0]:
!cd ocr_densenet/code/ocr;python main.py --phase train --resume ../../data/models-small/densenet/eval-16-1/best_f1score.ckpt

In [0]:
!cd ocr_densenet/code/ocr;python main.py --phase test --resume ../../data/models-small/densenet/eval-16-1/best_f1score.ckpt

best_f1score 0.999665101417
('len(train_filelist1)', 0)
('len(train_filelist2)', 0)

test test_files, epoch: 13
../../data/feats/test
 70% 7024/10000 [02:56<01:14, 39.87it/s]P5MUY42D.jpg
100% 10000/10000 [04:07<00:00, 40.46it/s]


### 根据 20_test_preds_max1.npy 中各个字符的概率,将不满足正则的数据进行重新整理，提高一个百分点

In [0]:
import json
import math
word_index_dict = json.load(open('/content/ocr_densenet/files/alphabet_index_dict.json'))
index_word_dict = { v:k for k,v in word_index_dict.items() }

import pandas as pd
import re
import numpy as np
probs = np.load("./ocr_densenet/data/result/21_test_preds_max1.npy",allow_pickle=True)
print(probs.shape)

def pst10(p):
  mid=[]
  for i in range(64):
    max = -4
    w = '*'
    for j in range(len(p[i])):
      if p[i][j]>max:
        max = p[i][j]
        w = index_word_dict[j]
    mid.append(w)
  print(''.join(mid))
  result=[]
  lx =0
  last = '*'
  for i in mid:
    if i != '*' and last=='*':
      result.append(i)
      last = i
      lx = 1
    elif i!= '*' and last!=i:
      result.append(i)
      last = i
      lx = 1
    elif i!='*' and last==i and lx >3:
      result.append(i)
      last = i
      lx = 1
    elif i!= '*' and last==i:
#       last = i
      lx = lx +1
    else:
      last = i
      lx = 1
  return ''.join(result)

def pgt10(p):
  result = []
  mid=[]
  score = []
  m_score= []
  last = '*'
  for i in range(64):
    max = -4
    w = '*'
    for j in range(len(p[i])):
      if p[i][j]>max:
        max = p[i][j]
        w = index_word_dict[j]
    mid.append(w)   
#     print(w,last,m_score,score)
    if w != '*':
      m_score.append(max)
    elif w == '*' and last!='*':
      score.append(m_score)
      m_score = []
    last = w
  mid = ''.join(mid)
  for i in range(10):
    mid = mid.replace('**','*')
  if mid[:1] == '*':
    mid = mid[1:]
  if mid[-1:] == '*':
    mid = mid[:-1]
  mids = mid.split('*') 
  
#   print(score)
  for index,m in enumerate(mids):
    dist = []
    r_dist = []
    scs = {}
    last = '*'
    st = 0
    lx = 0
    for ii,sm in enumerate(m):
      sco = score[index][ii]
      if sco<0:
          sco = math.exp(sco)
      if ii == 0 :
        dist.append(st)
        r_dist.append(sm)
        scs[st] = sco
        last = sm
      elif sm == last and lx < 4:
        lx = lx + 1
        scs[st] = scs[st] + sco
      elif lx>4:
        lx = 0
        st = st + 1
        last = sm
        dist.append(st)
        r_dist.append(sm)
        scs[st] = sco
      else:
        lx = 0
        st = st + 1
        last = sm
        dist.append(st)
        r_dist.append(sm)
        scs[st] = sco
#       if sm not in dist:
#         dist.append(sm)
#       if scs.has_key(sm):
#         sco = score[index][ii]
#         if score[index][ii]<0:
#           sco = math.exp(sco)
#         scs[sm] = scs[sm] + sco
#       else:
#         scs[sm] = score[index][ii]
    scs = sorted(scs.items(),key = lambda item:item[1],reverse=True)
    
    if len(m)<5:
      scs = dict(scs[:1])
      for i,d in enumerate(dist):
        if d in scs.keys():
          result.append(r_dist[i])
    elif len(m)<9:
      scs = dict(scs[:2])
      for i,d in enumerate(dist):
        if d in scs.keys():
          result.append(r_dist[i])
    elif len(m)<12:
      scs = dict(scs[:3])
      for i,d in enumerate(dist):
        if d in scs.keys():
          result.append(r_dist[i])
    else:
      scs = dict(scs[:4])
      for i,d in enumerate(dist):
        if d in scs.keys():
          result.append(r_dist[i])
  return  ''.join(result)    
#   print(mid)
#   print(score)
data = pd.read_csv('./ocr_densenet/data/result/21_test_result.csv')
i = 0 
for index, row in data.iterrows():
  name = row['name']
  label = row['label']
  prob = probs[index:index+1][0][0]
  if not re.match(r'\D(\D\d|\d\D)\d{7}',label) or len(label)!=10:
    r = pst10(prob)
    print(index,name,label,r)
    if len(r)>10: 
      rr = pgt10(prob) 
      print(index,name,label,r,rr)
      if len(rr)!=10:
        i = i + 1
        print('/******\n')
        print('bt......10',index,name,label,r,rr)
        print('\n******/')
        data.loc[data['name']==name,'label'] = rr[len(rr)-10:]
      else:
        data.loc[data['name']==name,'label'] = rr
    else:
      data.loc[data['name']==name,'label'] = r
print(i)
data.to_csv('./old_densent_rg_result.csv',index=False)
print(i)

(10000,)
*************CCC*77***WWW*7779999*22222222444*9999*222**********
(286, '10G7SLEY.jpg', 'C7W792492', u'C7W7922492')
*************FFF***PPP*44*777*0002222222*11*888*888*************
(347, '18E5AHRD.jpg', 'FP4702188', u'FP47022188')
***************HHH888**YYY*6666444*22222222999*777*555**********
(353, '1905G3Q4.jpg', 'H8Y642975', u'H8Y6422975')
*************PPPP**CCC*555*000*888*6662222*11*22222222**********
(366, '1BE8SPIU.jpg', 'PC5086212', u'PC50862122')
*************FFFF**XX*111*0002222222*888111*222888**************
(529, '1W8GLXRM.jpg', 'FX1028128', u'FX10228128')
*************ZZZZ*GGG*555*2222222*666*444999*888*666************
(607, '27G1FQHB.jpg', 'ZG5264986', u'ZG52264986')
******AA*****1*QQ*11**100Q******00*****4666*******Q*****588*****
(661, '2D9QKGRM.jpg', 'A64', u'A1Q110Q046Q58')
(661, '2D9QKGRM.jpg', 'A64', u'A1Q110Q046Q58', u'A1Q1006Q8')
/******

('bt......10', 661, '2D9QKGRM.jpg', 'A64', u'A1Q110Q046Q58', u'A1Q1006Q8')

******/
***************HHHH77***44*888*666*

### 可视化不同模型检测不一致的情况,重新剪裁图片

In [0]:
import os
from shutil import copyfile
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
def drow(img):
  img = Image.open('/content/test_cptn_result/'+img)
#   .convert('L')
#   img = img.resize((160,32), Image.ANTIALIAS)
  plt.imshow(img, cmap='gray')
  plt.show()
  
import pandas as pd
label1_data = pd.read_csv('./cptn-crnn/crnn/crnn_train_result_0606.csv')
label2_data = pd.read_csv('./densent_rg_result.csv')
label2_data['label']=label2_data['label'].map(str.strip)

i = 0
for index, row in label1_data.iterrows():
  name = row['name']
  label1 = row['label']
  try:
    label2 = label2_data[label2_data['name']==name]['label'].values[0].strip()
    if label1!=label2:
      if not os.path.exists(os.path.join('./train_cptn_result',name)):
              print('%s does not exist' % imagePath)
              continue
      copyfile(os.path.join('./train_cptn_result',name),'./train_cptn_result_repair/'+name)
      i = i +1
      print(index,row['name'],label1,label2,drow(row['name']))
  except:
    continue
print(i)